# Operations for DNS Sample Code


## Initial Login

In [1]:
from avi.sdk.avi_api import ApiSession
from requests.packages import urllib3
urllib3.disable_warnings()
import json


# Import custom libraries
from libs.aux import *

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:llw86y8tod6cseskya2rsvowanihjwle


### Step 0.- Select DNS VS to perform operations
Sometimes options might not be displayed. If so, stop the cell execution and restart. As last resort, create variables manually

```
dns_vs = "dns"
```

In [26]:
# Get Virtual Services containing that field (indication of a DNS-type VS)
query = {
    "fields": "name",
    "search": "(port,53)"
}

resp = api.get("virtualservice", params=query)
resp = json.loads(resp.text)["results"]

menu_options = []
i = 0
for item in resp:
    menu_options.insert(i, "VirtualService: "+item["name"])
    i = i +1

menu_title = "Please Select an option :"
selected_option = display_menu_from_list(menu_options, menu_title)

print()
if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m as VS to create your A records")
else:
    print("No valid option was selected.")

dns_vs_name = selected_option.split(": ")[1]

Please Select an option :
--------------------------------------------------------------
1. VirtualService: dns

You selected: VirtualService: dns as VS to create your A records


In [4]:
# Set DNS VS mannually 
# Uncomennt and populate both variables if interactive menu does not work
# dns_vs_name = "dns"

## STEP 1.- Collecting VS information

In [33]:
vs_data = extract_dns_records_from_vs(api, dns_vs_name)
print_dns_records(vs_data)

Getting information of virtualservice named:
['dns']

ID FQDN                      IP Address      TTL   Type
------------------------------------------------------------
0  test4.example.com         192.168.1.203   4     DNS_RECORD_A


## STEP 2.- Creating new DNS Record

In [ ]:
# Create a new object 
NEW_DNS_RECORD_FQDN="test4.example.com"
NEW_DNS_RECORD_IP_ADDRESS="192.168.1.203"
NEW_DNS_RECORD_TTL= 4
NEW_DNS_RECORD_TYPE="DNS_RECORD_A"

In [32]:
# Craft payload using above parameters
new_dns_record= {
    'fqdn': [NEW_DNS_RECORD_FQDN],
    'ip_address': [{
        'ip_address': {
            'addr': NEW_DNS_RECORD_IP_ADDRESS, 
            'type': 'V4'}
            }],
    'ttl': NEW_DNS_RECORD_TTL,
    'type': NEW_DNS_RECORD_TYPE}

# Insert new data
if 'static_dns_records' in vs_data:
    vs_data['static_dns_records'].append(new_dns_record)
else:
    vs_data['static_dns_records'] = [new_dns_record]

# Define PUT parameters
url_path="virtualservice/"+vs_data["uuid"]
body =  vs_data

#Send BODY information via PUT
resp = api.put (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
    print()
else:
    print('Error in modifying '+url_path+' :%s' % resp.text)


<Response [200]>
- Object virtualservice/virtualservice-7bb2c9c7-f08e-4911-8204-3cf512bf29d4 named dns modified OK



## DELETE existing Record

In [ ]:
vs_data = extract_dns_records_from_vs(api, dns_vs_name)
menu_options = []
i = 0
for item in vs_data["static_dns_records"]:
    menu_options.insert(i, "FQDN: "+item["fqdn"][0]+" IP_ADDRESS:"+item["ip_address"][0]["ip_address"]["addr"]+" TYPE:"+item["type"])
    i = i +1

menu_title = "Please Select an option :"
selected_option = display_menu_from_list(menu_options, menu_title)

print()
if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m to delete")
else:
    print("No valid option was selected.")

selected_record_fqdn = selected_option.split(": ")[1].split(" ")[0]

Getting information of virtualservice named:
['dns']

Please Select an option :
--------------------------------------------------------------
1. FQDN: test4.example.com IP_ADDRESS:192.168.1.203 TYPE:DNS_RECORD_A

You selected: FQDN: test4.example.com IP_ADDRESS:192.168.1.203 TYPE:DNS_RECORD_A to delete


In [29]:
# Remove entry with partitular FQDN
fqdn_to_remove = selected_record_fqdn
vs_data["static_dns_records"] = [
    record for record in vs_data["static_dns_records"]
    if record.get("fqdn", [None])[0] != fqdn_to_remove
]

# Define PUT parameters
url_path="virtualservice/"+vs_data["uuid"]
body =  vs_data

#Send BODY information via PUT
resp = api.put (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
    print()
else:
    print('Error in modifying '+url_path+' :%s' % resp.text)

# Printing new static entries
print ("printing modified table")
vs_data = extract_dns_records_from_vs(api, dns_vs_name)
print_dns_records(vs_data)

<Response [200]>
- Object virtualservice/virtualservice-7bb2c9c7-f08e-4911-8204-3cf512bf29d4 named dns modified OK

printing modified table
Getting information of virtualservice named:
['dns']

ID FQDN                      IP Address      TTL   Type
------------------------------------------------------------


## UPDATE Existing Record

In [35]:
vs_data = extract_dns_records_from_vs(api, dns_vs_name)
menu_options = []
i = 0
for item in vs_data["static_dns_records"]:
    menu_options.insert(i, "FQDN: "+item["fqdn"][0]+" IP_ADDRESS:"+item["ip_address"][0]["ip_address"]["addr"]+" TYPE:"+item["type"])
    i = i +1

menu_title = "Please Select an option :"
selected_option = display_menu_from_list(menu_options, menu_title)

print()
if selected_option:
    print("You selected: \033[1m"+selected_option+"\033[0m to delete")
else:
    print("No valid option was selected.")

selected_record_fqdn = selected_option.split(": ")[1].split(" ")[0]

Getting information of virtualservice named:
['dns']

Please Select an option :
--------------------------------------------------------------
1. FQDN: test222.example.com IP_ADDRESS:192.168.1.1 TYPE:DNS_RECORD_A

You selected: FQDN: test222.example.com IP_ADDRESS:192.168.1.1 TYPE:DNS_RECORD_A to delete


In [49]:
# Generate new payload
NEW_DNS_RECORD_FQDN=selected_record_fqdn
NEW_DNS_RECORD_IP_ADDRESS="192.168.1.211"
NEW_DNS_RECORD_TTL= 4
NEW_DNS_RECORD_TYPE="DNS_RECORD_A"

In [50]:
# Craft payload using above parameters
new_dns_record= {
    'fqdn': [selected_record_fqdn],
    'ip_address': [{
        'ip_address': {
            'addr': NEW_DNS_RECORD_IP_ADDRESS, 
            'type': 'V4'}
            }],
    'ttl': NEW_DNS_RECORD_TTL,
    'type': NEW_DNS_RECORD_TYPE}

vs_data = extract_dns_records_from_vs(api, dns_vs_name)
fqdn = new_dns_record["fqdn"][0]
existing_records = vs_data.get("static_dns_records", [])

vs_data["static_dns_records"] = [
    rec for rec in existing_records
    if fqdn not in rec.get("fqdn", [])
]

# Ahora puedes añadir el nuevo sin riesgo de duplicados
vs_data["static_dns_records"].append(new_dns_record)

# Define PUT parameters
url_path="virtualservice/"+vs_data["uuid"]
body =  vs_data

#Send BODY information via PUT
resp = api.put (url_path, data=json.dumps(body))

if resp.status_code in range(200, 299):
    print(resp)
    print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
    print()
else:
    print('Error in modifying '+url_path+' :%s' % resp.text)


# Printing new static entries
print ("printing modified table")
vs_data = extract_dns_records_from_vs(api, dns_vs_name)
print_dns_records(vs_data)

Getting information of virtualservice named:
['dns']

<Response [200]>
- Object virtualservice/virtualservice-7bb2c9c7-f08e-4911-8204-3cf512bf29d4 named dns modified OK

printing modified table
Getting information of virtualservice named:
['dns']

ID FQDN                      IP Address      TTL   Type
------------------------------------------------------------
0  test222.example.com       192.168.1.211   4     DNS_RECORD_A
